In [ ]:
from __future__ import print_function
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import base64
import json
import time
from IPython import display
from IPython.lib import backgroundjobs as bg

In [ ]:
def save_image(payload):
    """Save image as png file to display in Notebook later."""
    decoded = base64.b64decode(payload['image'])  # Decode image from json
    with open('test.png', 'wb') as f:
        f.write(decoded)
    return payload['timestamp'], payload['camera_id']

In [ ]:
def update_image():
    """Update jupyter display with image."""
    while True:
        time.sleep(3)
        image.reload()
        handle.update(image)

In [ ]:
# Show image in Notebook
image = display.Image(filename='test.png', width=300, height=200)
handle = display.display(image, display_id=True)

In [ ]:
%%capture
#    ^ Suppress output of Cell in Jupyter 

# Start background-job to update image
jobs = bg.BackgroundJobManager();
jobs.new('update_image()');

In [ ]:
# Listen to Kafka Stream

# Stream Settings
zkQuorum = 'localhost:2181'
topic = 'wordcounttopic'
ssc = StreamingContext(sc, 3)  # Update every 5 seconds
kvs = KafkaUtils.createStream(ssc, zkQuorum, "spark-streaming-consumer", {topic: 1})

# Transform stream payload
dstream = kvs.map(lambda x: json.loads(x[1]))  # transform json to object
images = dstream.map(lambda x: save_image(x))  # get image string & save

images.pprint(5)

# Actually start Streaming Context
ssc.start()  
ssc.awaitTermination()